# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter # esealy format and save excel file from python script

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2360304334900,
 'week52high': 150.04,
 'week52low': 55.8,
 'week52change': 0.7234985785953827,
 'sharesOutstanding': 16844940388,
 'float': 0,
 'avg10Volume': 133916163,
 'avg30Volume': 116722875,
 'day200MovingAvg': 123.55,
 'day50MovingAvg': 138.28,
 'employees': 0,
 'ttmEPS': 3.74,
 'ttmDividendRate': 0.8108260709976199,
 'dividendYield': 0.006045311168891038,
 'nextDividendDate': '2021-01-24',
 'exDividendDate': '2020-10-30',
 'nextEarningsDate': '2021-01-15',
 'peRatio': 36.43220897425476,
 'beta': 1.154027179021789,
 'maxChangePercent': 50.74946989462516,
 'year5ChangePercent': 5.173622547208617,
 'year2ChangePercent': 2.3216818643904276,
 'year1ChangePercent': 0.7677394391537156,
 'ytdChangePercent': 0.017897473876839484,
 'month6ChangePercent': 0.2809459468779096,
 'month3ChangePercent': 0.2434518631176247,
 'month1ChangePercent': 0.018014072158917656,
 'day30ChangePercent': 0.017555572045251887,
 'day5ChangePercent': -0.05942422647938

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [6]:
data['year1ChangePercent']

0.7677394391537156

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [18]:
final_data = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    #print(data)
    for tick in data :
        final_data = final_data.append(pd.Series([tick, data[tick]['price'], data[tick]['stats']['year1ChangePercent'],'N/A'],index = my_columns), ignore_index=True)
final_data
        

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,127.89,0.524633,N/A
1,AAL,17.14,-0.381569,N/A
2,AAP,156.28,0.181001,N/A
3,AAPL,139.24,0.774338,N/A
4,ABBV,107.07,0.350678,N/A
...,...,...,...,...
500,YUM,111.60,0.0246136,N/A
501,ZBH,165.90,0.0804817,N/A
502,ZBRA,422.93,0.7281,N/A
503,ZION,48.00,0.0450879,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [19]:
final_data.sort_values('One-Year Price Return',ascending=False,inplace = True)

In [22]:
final_data = final_data[:50]
final_data.reset_index(inplace=True)
final_data

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,39.08,2.31256,N/A
1,179,FCX,28.83,1.53601,N/A
2,345,NVDA,549.10,1.29989,N/A
3,387,PYPL,258.90,1.21644,N/A
4,23,ALB,173.30,1.19787,N/A
5,24,ALGN,574.72,1.18734,N/A
6,388,QCOM,167.04,0.992554,N/A
7,410,SIVB,484.43,0.959669,N/A
8,128,DE,312.90,0.951989,N/A
9,275,LB,44.60,0.948015,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [24]:
portfolio = 10000000
position_size = portfolio/len(final_data.index)
position_size

200000.0

In [26]:
for i in range(len(final_data.index)):
    final_data.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_data.loc[i,'Price'])
final_data


C:\ProgramData\Anaconda3\envs\python36\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,39.08,2.31256,5117
1,179,FCX,28.83,1.53601,6937
2,345,NVDA,549.10,1.29989,364
3,387,PYPL,258.90,1.21644,772
4,23,ALB,173.30,1.19787,1154
5,24,ALGN,574.72,1.18734,347
6,388,QCOM,167.04,0.992554,1197
7,410,SIVB,484.43,0.959669,412
8,128,DE,312.90,0.951989,639
9,275,LB,44.60,0.948015,4484


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [57]:
hdm_columns = [
    'Ticker', 
    'Price', 
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]
hqm_dataframe = pd.DataFrame(columns=hdm_columns)
#hqm_dataframe
for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    #print(data)
    for tick in data :
        hqm_dataframe = hqm_dataframe.append(
            pd.Series
            (
                [
                tick, 
                data[tick]['price'], 
                'N/A',
                data[tick]['stats']['year1ChangePercent'],
                'N/A',
                data[tick]['stats']['month6ChangePercent'],
                'N/A',
                data[tick]['stats']['month3ChangePercent'],
                'N/A',
                data[tick]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
                ],
                index = hdm_columns
            ), 
            ignore_index=True
        )
hqm_dataframe
hqm_dataframe.replace(to_replace=[None], value=0, inplace=True)

In [58]:
periods = ['One-Year','Six-Month','Three-Month','One-Month']

for raw in hqm_dataframe.index:
    for period in periods :
        percentile_col = f'{period} Return Percentile'
        change_col = f'{period} Price Return'
        hqm_dataframe.loc[raw,percentile_col] = stats.percentileofscore(hqm_dataframe[change_col],hqm_dataframe.loc[raw,change_col])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,124.330,N/A,0.514692,87.9208,0.291701,66.3366,0.176642,53.6634,0.042773,69.3069,N/A
1,AAL,17.490,N/A,-0.396612,1.78218,0.513999,87.7228,0.498695,93.8614,0.052189,73.4653,N/A
2,AAP,157.500,N/A,0.178837,58.4158,0.030586,20,0.019917,16.2376,-0.022497,33.4653,N/A
3,AAPL,135.390,N/A,0.781423,95.6436,0.283060,64.7525,0.244748,66.5347,0.017746,53.8614,N/A
4,ABBV,108.932,N/A,0.358275,79.4059,0.119659,35.2475,0.189538,55.8416,-0.023287,32.2772,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.030,N/A,0.024694,36.8317,0.185307,48.3168,0.122449,40.9901,-0.021726,34.2574,N/A
501,ZBH,162.010,N/A,0.082296,45.3465,0.181338,46.9307,0.201894,59.604,0.028743,59.4059,N/A
502,ZBRA,419.620,N/A,0.718117,94.6535,0.454316,83.5644,0.386997,85.3465,0.055005,75.4455,N/A
503,ZION,47.370,N/A,0.045630,39.604,0.442453,82.1782,0.393781,85.7426,0.054354,74.8515,N/A


In [59]:
for i in range(len(hqm_dataframe.index)):
    hqm_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,124.330,1608,0.514692,87.9208,0.291701,66.3366,0.176642,53.6634,0.042773,69.3069,N/A
1,AAL,17.490,11435,-0.396612,1.78218,0.513999,87.7228,0.498695,93.8614,0.052189,73.4653,N/A
2,AAP,157.500,1269,0.178837,58.4158,0.030586,20,0.019917,16.2376,-0.022497,33.4653,N/A
3,AAPL,135.390,1477,0.781423,95.6436,0.283060,64.7525,0.244748,66.5347,0.017746,53.8614,N/A
4,ABBV,108.932,1836,0.358275,79.4059,0.119659,35.2475,0.189538,55.8416,-0.023287,32.2772,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.030,1834,0.024694,36.8317,0.185307,48.3168,0.122449,40.9901,-0.021726,34.2574,N/A
501,ZBH,162.010,1234,0.082296,45.3465,0.181338,46.9307,0.201894,59.604,0.028743,59.4059,N/A
502,ZBRA,419.620,476,0.718117,94.6535,0.454316,83.5644,0.386997,85.3465,0.055005,75.4455,N/A
503,ZION,47.370,4222,0.045630,39.604,0.442453,82.1782,0.393781,85.7426,0.054354,74.8515,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

- Already DONE ABOVE

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [55]:
from statistics import mean

In [61]:
hqm_dataframe.loc(2,'One-Year Return Percentile')

TypeError: __call__() takes from 1 to 2 positional arguments but 3 were given

In [62]:
for raw in hqm_dataframe.index:
    momentum_percentile = []
    for period in periods:
        momentum_percentile.append(hqm_dataframe.loc[raw,f'{period} Return Percentile'])
    hqm_dataframe.loc[raw,'HQM Score'] = mean(momentum_percentile)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,124.330,1608,0.514692,87.9208,0.291701,66.3366,0.176642,53.6634,0.042773,69.3069,69.3069
1,AAL,17.490,11435,-0.396612,1.78218,0.513999,87.7228,0.498695,93.8614,0.052189,73.4653,64.2079
2,AAP,157.500,1269,0.178837,58.4158,0.030586,20,0.019917,16.2376,-0.022497,33.4653,32.0297
3,AAPL,135.390,1477,0.781423,95.6436,0.283060,64.7525,0.244748,66.5347,0.017746,53.8614,70.198
4,ABBV,108.932,1836,0.358275,79.4059,0.119659,35.2475,0.189538,55.8416,-0.023287,32.2772,50.6931
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.030,1834,0.024694,36.8317,0.185307,48.3168,0.122449,40.9901,-0.021726,34.2574,40.099
501,ZBH,162.010,1234,0.082296,45.3465,0.181338,46.9307,0.201894,59.604,0.028743,59.4059,52.8218
502,ZBRA,419.620,476,0.718117,94.6535,0.454316,83.5644,0.386997,85.3465,0.055005,75.4455,84.7525
503,ZION,47.370,4222,0.045630,39.604,0.442453,82.1782,0.393781,85.7426,0.054354,74.8515,70.5941


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [66]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True,drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,165.260,1210,1.176473,99.0099,1.113992,99.0099,0.772221,98.0198,0.153172,95.4455,97.8713
1,SIVB,479.246,417,0.985427,98.6139,1.091430,98.8119,0.544209,95.8416,0.215407,97.8218,97.7723
2,VIAC,51.510,3882,0.547396,89.505,0.983220,98.4158,0.725017,97.8218,0.350959,100,96.4356
3,GM,55.800,3584,0.622339,91.2871,1.134860,99.2079,0.533570,95.4455,0.271974,99.2079,96.2871
4,AMAT,103.422,1933,0.841700,96.0396,0.624643,92.6733,0.723292,97.6238,0.203492,97.4257,95.9406
5,FCX,29.468,6787,1.548807,99.802,1.219327,99.4059,0.556224,96.0396,0.082410,84.9505,95.0495
6,DISCA,40.370,4954,0.378310,81.3861,0.908550,97.6238,0.910319,99.2079,0.345309,99.802,94.505
7,KLAC,304.263,657,0.849929,96.4356,0.516560,88.1188,0.505528,94.2574,0.157426,95.6436,93.6139
8,LB,45.361,4409,0.928280,98.0198,0.844615,97.2277,0.341336,80.9901,0.189615,96.6337,93.2178
9,IVZ,21.580,9267,0.315523,75.0495,1.231662,99.604,0.609848,97.2277,0.236364,98.6139,92.6238


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

- Already DONE ABOVE

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: